# Prompt Templates

---

Hello everyone! In this notebook, we will explore the concept of "Prompt Templates," which you will frequently encounter in LangChain projects.

The goal of this module is to master:
1. Prompt Templates
2. Output Parsers

## Load environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv()

## By default, load_dotenv() will assign environment variables into os.environ, like following code:
## See environment variables in .env file
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] =  os.getenv('LANGCHAIN_API_KEY')
# os.environ["OPENAPI_KEY"] =  os.getenv('OPENAI_KEY')

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

# Chat Prompt Template

In Module 1, "Language Model," we passed prompt as a string or a list of messages directly into the language model.

However, in most cases, the input to a language model is constructed from a combination of user input and application logic. This application logic typically takes the raw user input and transforms it into a list of messages that are ready to be passed to the language model.

**PromptTemplates** are a concept in LangChain designed to facilitate this transformation. They take in raw user input and return data (a prompt) that is ready to be passed into a language model.

Let's create a `PromptTemplate` that accepts a user variable called `country`

In [17]:
from langchain_core.prompts import ChatPromptTemplate   

template = "What is capital city of {country}?"

prompt_template = ChatPromptTemplate.from_template(template)

# .invoke method will genereate prompt ready to be passed to the language model
message_prompt = prompt_template.invoke({"country": "Italia"})

# prompt_template.input_schema

message_prompt

pydantic.v1.main.PromptInput

In [2]:
message_prompt.to_messages() # => this is prompt ready to be passed into a language model

[HumanMessage(content='What is capital city of Italia?')]

Let's create a `PromptTemplate` that will generate a list of messages. It will accept two arguments:

- `language`: The target language of translation
- `text`: The tex to be translated

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Translate the following into {language}:"),
        ("user", "{text}"),
    ]
)   
message_prompt = prompt_template.invoke({"language": "italian", "text": "hi"})

message_prompt.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

## 2 - Alternative Ways To Build Prompt Template

You can pass any Message-like formats supported by `ChatPromptTemplate.from_messages()` directly to `ChatPromptTemplate()` init. 

#### 1. **`string`
### 2. **`list of tuple`**

#### 3. **`dictionary`**
   - **Example**:
     ```python
     template_dict = {
         "system_message": "You are a helpful assistant.",
         "user_message": "Hello, {name}!"
     }
     prompt = ChatPromptTemplate.from_message(template_dict)
     ```

#### 4. **`json`**
   - **Example**:
     ```python
     json_str = '{"system_message": "You are a helpful assistant.", "user_message": "Hello, {name}!"}'
     prompt = ChatPromptTemplate.from_json(json_str)
     ```

In [15]:
prompt_template = ChatPromptTemplate(
    {
        "system_message": "Translate the following into {language}:",
        "user_message": "{text}",
    }
)   
message_prompt = prompt_template.invoke({"language": "italian", "text": "hi"})

message_prompt.to_messages()

[HumanMessage(content='system_message'), HumanMessage(content='user_message')]

## Output Parser

One useful and frequently used concept in langchain is output parser. It allows you to parse a AI response object into a desired format, for example string. See code below:

In [16]:
from langchain_core.output_parsers import StrOutputParser  
from langchain_core.messages import AIMessage

# typical AI respond
message = AIMessage(content="Hi Jim. How can I assist you today?")

parser = StrOutputParser()

parser.invoke(message)

'Hi Jim. How can I assist you today?'

We can also format output to different format, such as `JsonOutputParser`, `YamlOututParser`, or `CustomOutputParser`